# **PROGETTO SOCIAL COMPUTING**

STUDENTI: Marco Tateo, Leonardo Scandino, Leo Baric, Matteo Palomba

## Impostazione

In [ ]:
import os 
import json
import pprint
import tweepy
import networkx as nx
import pandas as pd
import numpy as np
from random import *
from pyvis.network import Network
from networkx.algorithms.approximation import clique
import scipy
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats import pearsonr, kendalltau

In [ ]:
api_key = "****"
api_secret = "****"
access_token = "****"
access_secret = "****"
bearer_token = "****"

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

In [ ]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.pakendalltaupath):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

### 1. Scaricate utenti followers (api.followers) e utenti following (api.friends) di questi cinque account (i dati numerici potrebbero subire minime variazioni)

##### Follower

In [ ]:
prof = ["@KevinRoitero"]     #nel array prof si possono mettere tutti i prof per scaricare i follower di tutti e cinque i prof automaticamente
followers = {}               #abbiamo diviso il lavoro per velocizzare il processo 

for user in prof:
    
    print(f"Processing User {user}")
    
    followers_of_user = [] 
    
    for item in tweepy.Cursor(
        api.followers, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
    ).items():
        
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        
        if found_follower not in followers_of_user:
            followers_of_user.append(found_follower)
            
    followers = followers_of_user
    print(f"Found {len(followers)} followers for user {user}")
    serialize_json(f"{data_folder}/Follower", f"{user}1_followers.json", followers) #salva in un file .json i follower del prof in input

##### Followings

In [ ]:
prof = ["@KevinRoitero"]   #nel array prof si possono mettere tutti i prof per scaricare i followings di tutti e cinque i prof automaticamente
followings = {}             #abbiamo diviso il lavoro per velocizzare il processo

for user in prof:
    
    print(f"Processing User {user}")
    
    followings_of_user = [] 
    
    for item in tweepy.Cursor(
        api.friends, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
    ).items():
        
        json_data = item._json
        
        found_following = {}
        found_following["id"] = json_data["id"]
        found_following["name"] = json_data["name"]
        found_following["screen_name"] = json_data["screen_name"]
        found_following["location"] = json_data["location"]
        
        if found_following not in followings_of_user:
            followings_of_user.append(found_following)
            
    followings = followings_of_user
    print(f"Found {len(followings)} followings for user {user}")
    serialize_json(f"{data_folder}/Following", f"{user}_followings.json", followings)  #salva in un file .json tutti i folliwings del prof

##### Uniamo i file json in un unico file json che contiene i follower di tutti 5 professori

In [ ]:
prof = ["@mizzaro", "@damiano10", "@Miccighel_", "@eglu81", "@KevinRoitero"]

all_followers = {}
temp = []

for user in prof:
    file = read_json(f"{data_folder}/Follower/{user}_followers.json")
    temp.extend(file)

all_followers = temp
serialize_json(f"{data_folder}/Follower","all_prof_followers.json",all_followers)   #tutti follower sono nel file all_prof_followers.json

##### Uniamo i file json in un unico file json che contiene i followings di tutti 5 professori

In [ ]:
users = ["@mizzaro", "@damiano10", "@Miccighel_", "@eglu81", "@KevinRoitero"]

all_followings = {}
temp = []

for user in users:
    file = read_json(f"{data_folder}/Following/{user}_followings.json")
    temp.extend(file)
    
all_followings = temp
serialize_json(f"{data_folder}/Following","all_prof_followings.json",all_followings)  #tutti followings sono nel file all_prof_followers.json

### 2. Scegliete 5 utenti followers a caso tra quelli di ciascuno dei cinque account e scaricate ulteriori 10 utenti followers (followers dei followers)


##### Dalla lista di ogni prof prende 5 random follower

In [ ]:
users = ["@mizzaro", "@damiano10", "@Miccighel_", "@eglu81", "@KevinRoitero"]
all_random = {}
random_followers = []
for user in users:
    
    followers_user = read_json(f"{data_folder}/Follower/{user}_followers.json")
    
    count = len(followers_user)

    for i in range (5):
        num = randint(0,count)
        single_random_follower = (followers_user[num])
        random_followers.append(single_random_follower)
        
all_random = random_followers        
        
serialize_json(f"{data_folder}/Random follower", "random_name.json", all_random) #random_name.json contiene i random follower dei prof

##### Per ognuno dei random follower scarica 10 follower

In [ ]:
random_name = read_json(f"{data_folder}/Random Follower/random_name.json")

followers_of_random = []

for random in random_name:
        
        print(f"Processing random User {random['screen_name']}")

        for item in tweepy.Cursor(
            api.followers, 
            screen_name=random['screen_name'], 
            skip_status=True, 
            include_user_entities=False
        ).items(10):

            json_data = item._json

            found_follower = {}
            found_follower["id"] = json_data["id"]
            found_follower["name"] = json_data["name"]
            found_follower["screen_name"] = json_data["screen_name"]
            found_follower["location"] = json_data["location"]
            
            if found_follower not in followers_of_random:
                followers_of_random.append(found_follower)

serialize_json(f"{data_folder}/Random follower", "followers_random.json", followers_of_random)  #followers_random.json contiene i follower dei random

### 3. Scegliete 5 utenti following a caso tra quelli di ciascuno dei cinque account e scaricate ulteriori 10 utenti following (following dei following)

##### Dalla lista di ogni prof prende 5 random followings

In [ ]:
users = ["@mizzaro", "@damiano10", "@Miccighel_", "@eglu81", "@KevinRoitero"]
all_random = {}
random_followings = []
for user in users:
    
    followings_user = read_json(f"{data_folder}/Following/{user}_followings.json")
    
    count = len(followings_user)

    for i in range (5):
        num = randint(0,count)
        single_random_following = (followings_user[num])
        random_followings.append(single_random_following)
        
all_random = random_followings        
        
serialize_json(f"{data_folder}/Random following", "random_name.json", all_random) #random_name.json contiene i random follower e followings dei prof

##### Per ognuno dei random followings scarica 10 followings

In [ ]:
random_name = read_json(f"{data_folder}/Random Following/random_name.json")

followings_of_random = []

for random in random_name:
        
        print(f"Processing random User {random['screen_name']}")

        for item in tweepy.Cursor(
            api.friends, 
            screen_name=random['screen_name'], 
            skip_status=True, 
            include_user_entities=False
        ).items(10):

            json_data = item._json

            found_following = {}
            found_following["id"] = json_data["id"]
            found_following["name"] = json_data["name"]
            found_following["screen_name"] = json_data["screen_name"]
            found_following["location"] = json_data["location"]
            
            if found_following not in followings_of_random:
                followings_of_random.append(found_following)

serialize_json(f"{data_folder}/Random following", "followings_random.json", followings_of_random)  #followings_random.json contiene i followings dei random

### 4. Scaricare i dettagli del profilo di tutti gli utenti recuperati (api.get_user)

##### Scarica i detagli dei profili dei prof

In [ ]:
users = ["@mizzaro", "@damiano10", "@Miccighel_", "@eglu81", "@KevinRoitero"]

prof = []
for user in users:
    
    details = api.get_user(user)
    json_data = details._json
    profile_data = {}
    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]
    if profile_data not in prof:
        prof.append(profile_data)
    
serialize_json(data_folder, "details_prof.json", prof)   #details_prof.json contiene i detagli degli account dei prof

##### Scarica i detagli dei follower dei prof

In [ ]:
lista = read_json(f"{data_folder}/all_prof_followers.json")
follower = []

for user in lista:
    
    id = user['id']
    details = api.get_user(user_id = id)
    json_data = details._json
    profile_data = {}
    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]
    if profile_data not in follower
        follower.append(profile_data)
    
serialize_json(data_folder, "details_prof_followers.json", follower) #details_prof_followers.json contiene i detagli dei follower dei prof

##### Scarica i detagli dei followings dei prof

In [ ]:
lista = read_json(f"{data_folder}/all_prof_followings.json")
followings = []

for user in lista:
    
    id = user['id']
    details = api.get_user(user_id = id)
    json_data = details._json
    profile_data = {}
    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]
    if profile_data not in followings:
        followings.append(profile_data)
    
serialize_json(data_folder, "details_prof_followings.json", followings) #details_prof_followings.json contiene detagli dei followings dei prof

##### Scarica i detagli dei follower dei random

In [ ]:
users = read_json(f"{data_folder}/Random follower/followers_random.json")

details = []
for user in users:
    r = api.get_user(user['screen_name'])
    json_data = r._json
    profile_data = {}
    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]
    if profile_data not in details:
        details.append(profile_data)
    
serialize_json(f"{data_folder}/Random follower", "details_followers_random.json", details)  #details_followers_random.json contiene i detagli
                                                                                            #dei follower dei random

##### Scarica i detagli dei followings dei random

In [ ]:
users = read_json(f"{data_folder}/Random following/followings_random.json")

details = []
for user in users:
    r = api.get_user(user['screen_name'])
    json_data = r._json
    profile_data = {}
    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]
    #if profile_data not in details:
    details.append(profile_data)
    
serialize_json(f"{data_folder}/Random following", "details_followings_random.json", details)  #details_followings_random.json contiene i 
                                                                                              #detagli dei followings dei random

### 5. Costruite la rete sociale (grafo):

#### -Inserite l’id di ciascun utente come identificatore del nodo
#### -Inserite i dettagli del profilo di ciascun utente come attributi del nodo
#### -Per ogni nodo, aggiungete un attributo con il numero di follower individuati
#### -Inserite i membri del vostro gruppo come attributi del grafo

In [ ]:
graph = nx.Graph()

#crea nodi per i professori 
prof = read_json(f"{data_folder}/details_prof.json")
for nodo in prof:
    graph.add_node(nodo['id'], name = nodo['name'], screen_name = nodo['screen_name'], location = nodo['location'], description = nodo['description'], followers_count = nodo['followers_count'])

#crea nodi per i follower dei prof
all_followers = read_json(f"{data_folder}/Follower/details_all_followers.json")
for nodo in all_followers:
    graph.add_node(nodo['id'], name = nodo['name'], screen_name = nodo['screen_name'], location = nodo['location'], description = nodo['description'], followers_count = nodo['followers_count'])

#crea nodi per i followings dei prof
all_followings = read_json(f"{data_folder}/Following/details_all_followings.json")
for nodo in all_followings:
    graph.add_node(nodo['id'], name = nodo['name'], screen_name = nodo['screen_name'], location = nodo['location'], description = nodo['description'], followers_count = nodo['followers_count'])
    
#crea nodi per i random follower
followers_random = read_json(f"{data_folder}/Random follower/details_followers_random.json")
for nodo in followers_random:
    graph.add_node(nodo['id'], name = nodo['name'], screen_name = nodo['screen_name'], location = nodo['location'], description = nodo['description'], followers_count = nodo['followers_count'])
    
#crea nodi per i random followings    
followings_random = read_json(f"{data_folder}/Random following/details_followings_random.json")
for nodo in followings_random:
    graph.add_node(nodo['id'], name = nodo['name'], screen_name = nodo['screen_name'], location = nodo['location'], description = nodo['description'], followers_count = nodo['followers_count'])

graph.graph["members"] = ["Marco Tateo, Leonardo Scandino, Leo Baric, Matteo Palomba"]

##### Prima di creare gli archi uniamo i file json di tutti follower, following, random follower e random following in un unico file json

In [ ]:
all_user = {}
temp = []

temp.extend(prof)
temp.extend(all_followers)
temp.extend(all_followings)
temp.extend(followers_random)
temp.extend(followings_random)


all_user = temp

serialize_json(data_folder, "all_user.json", all_user)   #all_user.json contiene tutti user scaricati 

#### -Ogni arco rappresenta una relazione follows tra due utenti
#### -La presenza della relazione va verificata tra tutti gli utenti scaricati ed i cinque account iniziali (api.show_friendship)


##### Crea archi tra ogni prof e i suoi follower e following

In [ ]:
file = read_json(f"{data_folder}/all_user.json")     #anche qui potevamo fare un loop che automaticamente crea archi per tutti i prof
                                                     #ma per questioni di tempo abbiamo diviso il lavoro tra i membri del gruppo
#mizzaro -> 18932422
#damiano-> 132646210
#micchighel -> 15750573
#eglu -> 19659370
#kevin -> 3036907250

for nodo in file:
    try:        #try per evitare l'errore che ci veniva se uno dei due user era privato o ha terminato il suo profilo
        friendship = api.show_friendship(source_id = "18932422", target_id = f"{nodo['id']}")
        if (friendship[0].following) or (friendship[0].followed_by):
            graph.add_edge(friendship[0].id , friendship[1].id)
    except:
        pass

nx.write_gpickle(graph, "data/graph_mizzaro.pkl")     #graph_mizzaro.pkl contiene tutti gli nodi e solo gli archi tra mizzaro i suoi follower/followings

##### Un modo alternativo per creare gli archi tra i professori e i loro follower e following senza utilizzare api.show_friendship()

In [ ]:
#Metodo non utilizzato nel progetto
#Scandisce i file json de follower e followings di ogni professore e crea archi 

#creazione archi per @eglu81
follower = read_json("data/Follower/@eglu81_followers.json")
following = read_json("data/Following/@eglu81_followings.json")

for nodo in following:
    graph.add_edge(19659370 , nodo['id'])

for nodo in follower:
    graph.add_edge(19659370  ,nodo['id'] )
print(graph.number_of_nodes())
print(graph.number_of_edges())


#creazione archi per @KEvinRoitero
follower = read_json("data/Follower/@KevinRoitero_followers.json")
following = read_json("data/Following/@KevinRoitero_followings.json")

for nodo in following:
    graph.add_edge(3036907250 , nodo['id'])

for nodo in follower:
    graph.add_edge(3036907250  ,nodo['id'] )
print(graph.number_of_nodes())
print(graph.number_of_edges())


#creazione archi per @Mizzaro
follower = read_json("data/Follower/@Mizzaro_followers.json")
following = read_json("data/Following/@Mizzaro_followings.json")

for nodo in following:
    graph.add_edge(18932422 , nodo['id'])

for nodo in follower:
    graph.add_edge(18932422  ,nodo['id'] )
print(graph.number_of_nodes())
print(graph.number_of_edges())


#creazione archi per @damiano10
follower = read_json("data/Follower/@damiano10_followers.json")
following = read_json("data/Following/@damiano10_followings.json")

for nodo in following:
    graph.add_edge(132646210 , nodo['id'])

for nodo in follower:
    graph.add_edge(132646210  ,nodo['id'] )
print(graph.number_of_nodes())
print(graph.number_of_edges())


#creazione archi per @Miccighel_
follower = read_json("data/Follower/@Miccighel__followers.json")
following = read_json("data/Following/@Miccighel__followings.json")

for nodo in following:
    graph.add_edge(15750573 , nodo['id'])

for nodo in follower:
    graph.add_edge(15750573  ,nodo['id'] )
print(graph.number_of_nodes())
print(graph.number_of_edges())

nx.write_gpickle(graph, "data/grafo_tutti_prof_.pkl")    

##### Crea archi tra i random 

In [ ]:
file = read_json(f"{data_folder}/all_user_random.json")
all_name_random = read_json(f"{data_folder}/all_name_random.json")

My_array = all_name_random[0:10]               #nella lista all_name_random ci sono 50 user e qui abbiamo diviso il lavoro in 5 parti per 
                                               #velocizzare l'esecuzione
for s_nodo in My_array:
    for t_nodo in file:
        try:          #try per evitare l'errore che ci veniva se uno dei due user era privato o ha terminato il suo profilo
            friendship = api.show_friendship(source_id = f"{s_nodo['id']}", target_id = f"{t_nodo['id']}")
            if (friendship[0].following) or (friendship[0].followed_by):
                graph.add_edge(friendship[0].id , friendship[1].id)
        except:
            pass
                       
nx.write_gpickle(graph, "data/graph_random_0-10.pkl")

##### Uniamo i grafi in un unico grafo con tutti nodi e archi

In [ ]:
graphMizzaro = nx.read_gpickle("data/graph_mizzaro.pkl")
graph.add_edges_from(graphMizzaro.edges)

graphDamiano = nx.read_gpickle("data/graph_damiano10.pkl")
graph.add_edges_from(graphDamiano.edges)

graphMiccighel = nx.read_gpickle("data/graph_micchighel.pkl")
graph.add_edges_from(graphMiccighel.edges)

graphEglu = nx.read_gpickle("data/graph_eglu.pkl")
graph.add_edges_from(graphEglu.edges)

graphKevin = nx.read_gpickle("data/graph_kevin.pkl")
graph.add_edges_from(graphKevin.edges)

graph0_10 = nx.read_gpickle(f"{data_folder}/graph_random_1-10.pkl")
graph.add_edges_from(graph0_10.edges)

graph10_20 = nx.read_gpickle(f"{data_folder}/graph_random_10-20.pkl")
graph.add_edges_from(graph10_20.edges)

graph20_30 = nx.read_gpickle(f"{data_folder}/graph_random_20-30.pkl")
graph.add_edges_from(graph20_30.edges)

graph30_33 = nx.read_gpickle(f"{data_folder}/graph_random_30-33.pkl")
graph.add_edges_from(graph30_33.edges)

graph33_36 = nx.read_gpickle(f"{data_folder}/graph_random_33-36.pkl")
graph.add_edges_from(graph33_36.edges)

graph36_40 = nx.read_gpickle(f"{data_folder}/graph_random_36-40.pkl")
graph.add_edges_from(graph36_40.edges)

graph40_50 = nx.read_gpickle(f"{data_folder}/graph_random_40-50.pkl")
graph.add_edges_from(graph40_50.edges)

nx.write_gpickle(graph, f"{data_folder}/final_graph.pkl")     #raccolto tutti gli archi in un unico grafo lo salviamo in final_graph.pkl

##### Alcuni nodi risultano isolati a causa del try/except percio a questo puno li rimuoviamo

In [ ]:
graph.remove_nodes_from(list(nx.isolates(graph)))

### 6. Producete una visualizzazione interattiva del grafo usando pyvis

In [ ]:
nt = Network( #per pagina html
    height="100%", 
    width="100%", 
    bgcolor="#222222", 
    font_color="white",
    heading="final_graph"
)
nt.barnes_hut() #prepara il layout 3D
nt.from_nx(final_graph) 
neighbor_map = nt.get_adj_list() #lista di adiacenza, vicinato
for node in nt.nodes:
    node["value"] = len(neighbor_map[node["id"]]) #imposto il peso del nodo


nt.show("final_graph_view.html")

### 7. Verificate se il grafo:

#### -è connesso (is_connected)

In [ ]:
if (nx.is_connected(final_graph)):
    print("Questo grafo è connesso")
else:
    print("Questo grafo NON è connesso")

#### -è bipartito (is_bipartite)

In [ ]:
if (nx.is_bipartite(final_graph)):
    print("Questo grafo è bipartito")
else:
    print("Questo grafo NON è bipartito")

### 8. Misurate le seguenti distanze sul grafo:

#### -Centro (center)

In [ ]:
center = nx.center(final_graph)
print(center)

#### -Diametro (diameter)

In [ ]:
diameter = nx.diameter(final_graph)
print(diameter)

#### -Raggio (radius)

In [ ]:
radius = nx.radius(final_graph)
print(radius)

### 9. Calcolate le seguenti misure di centralità sul grafo:

#### -Betweenness centrality (betweenness_centrality)

In [ ]:
bet_cent = nx.betweenness_centrality(final_graph)

draw(
    final_graph, 
    nx.spring_layout(final_graph), 
    nx.betweenness_centrality(final_graph), 
    'Betweenness Centrality'
)

#### -Closeness centrality (closeness_centrality)

In [ ]:
clo_cent = nx.closeness_centrality(final_graph)

draw(
    final_graph, 
    nx.spring_layout(final_graph), 
    nx.closeness_centrality(final_graph), 
    'closeness_centrality'
)

#### -Degree centrality (degree_centrality)

In [ ]:
deg_cent = nx.degree_centrality(final_graph)

draw(
    final_graph, 
    nx.spring_layout(final_graph), 
    nx.degree_centrality(final_graph), 
    'Degree Centrality'
)

#### -In-degree centrality (in_degree_centrality)

In [ ]:
final_graph_direct = final_graph.to_directed()   #per calcolare la in e out degree centrality il grafo deve essere diretto

in_deg_cent = nx.in_degree_centrality(final_graph_direct)

draw(
    final_graph_direct, 
    nx.spring_layout(final_graph_direct), 
    nx.in_degree_centrality(final_graph_direct), 
    'In Degree Centrality'
)

#### -Out-degree centrality (out_degree_centrality)

In [ ]:
out_deg_cent = nx.out_degree_centrality(final_graph_direct)

draw(
    final_graph_direct, 
    nx.spring_layout(final_graph_direct), 
    nx.in_degree_centrality(final_graph_direct), 
    'Out Degree Centrality'
)

#### -Page Rank (pagerank)

In [ ]:
page_rank = nx.pagerank(final_graph_direct)

draw(
    final_graph_direct, 
    nx.spring_layout(final_graph_direct), 
    nx.pagerank(final_graph_direct), 
    'Page Rank'
)

#### -HITS (hits)

In [ ]:
hits = nx.hits(final_graph_direct)

### 10. Generate un sottografo del grafo (ego_graph) e calcolate:

In [ ]:
kevin_graph = nx.ego_graph(final_graph, 3036907250, radius=1) 

##### Impostazioni per la visualizzazione del sottografo di Kevin

In [ ]:
color_map = []
for node in kevin_graph:
       if node  == 3036907250:
            color_map.append('red')
       else:
            color_map.append('black')  
        
d = dict(kevin_graph.degree)

rcParams['figure.figsize'] = 20, 20 
nx.draw_networkx(
    kevin_graph, 
    pos=nx.spring_layout(kevin_graph),
    node_color= color_map,
    edge_color="#a9a9a9",
    edge_cmap=plt.cm.Blues,
    node_size = [v * 30 for v in d.values()],
    alpha = 0.5,
    width=1,
    with_labels= False
)

plt.savefig(f"{data_folder}/Kevin_graph.pdf")
plt.show()
plt.close()

#### Cricca massima (max_clique)

In [ ]:
max_cl = clique.max_clique(kevin_graph)
clique_subgraph = kevin_graph.subgraph(max_cl) 
layout = nx.spring_layout(kevin_graph)
nx.draw_networkx(clique_subgraph, pos=layout)
plt.show()
plt.close()

#### -Dimensione della cricca massima (large_clique_size)

In [ ]:
large_clique_size = clique.large_clique_size(kevin_graph)
print(large_clique_size)

### 11. Calcolate la copertura minima degli archi (min_edge_cover) del grafo

In [ ]:
min_edge_cover = nx.min_edge_cover(final_graph)
print(min_edge_cover)

### 12. Calcolate i seguenti coefficienti per stimare la “small-world-ness” del grafo:

#### -Coefficiente omega (omega)

In [ ]:
omega = nx.omega(final_graph, niter = 10, nrand = 3)
print(omega)

#### -Coefficiente sigma (sigma)

In [ ]:
sigma = nx.sigma(final_graph, niter = 10, nrand = 3)
print(sigma)

### 13. Calcolare la correlazione di Pearson Rho e di Kendall Tau fra le misure di centralità; riportare il risultato in due tabelle

In [ ]:
centralità = [bet_cent, clo_cent, deg_cent, in_deg_cent, out_deg_cent, page_rank, hits[0], hits[1]]

In [ ]:
#coefficienti di Pearson Rho
pears_coef_values = []
for i in range (8):
    for t in range (8):
        if t > i:
            val = scipy.stats.pearsonr(list(centralità[i].values()), list(centralità[t].values()))
            pears_coef_values.append(val)

In [ ]:
#coefficienti di Kendall Tau
kendal_coef_values = []
for i in range (8):
    for t in range (8):
        if t > i:
            val = scipy.stats.kendalltau(list(centralità[i].values()), list(centralità[t].values()))
            kendal_coef_values.append(val)